In [1]:
from google.colab import drive
drive.mount('/content/drive/')
!python --version

Mounted at /content/drive/
Python 3.10.12


In [2]:
cd /content/drive/MyDrive/fake_news_detection

/content/drive/MyDrive/fake_news_detection


In [3]:
!pip install mxnet
!pip install gluonnlp==0.8.1 transformers==4.22.0
!pip install mosestokenizer
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 35.1 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.4/236.4 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 64.7 MB/s eta 0:00:00
  Created wheel for gluonnlp: filename=gluonnlp-0.8.1-py3-none-any.whl size=293516 sha256=3f0efb6feb64d54c498bbe59b7bd99a327330f5f00884e182ac011a604f24d69
  Stored in directory: /root/.cache/pip/wh

In [4]:
import pandas as pd
import jsonlines

test_path = './data/liar_plus_test.tsv'
test_df = pd.read_csv(test_path, sep="\t", header=None)
test_df = test_df.fillna(0)
test = test_df.values

##get data we need from dataset
labels = [test[i][2] for i in range(len(test))]
statements = [test[i][3] for i in range(len(test))]
justification = [test[i][15] for i in range(len(test))]

##exclude the rows which justification(evidence) doesn't exist
None_index_in_justification = [32, 101, 871, 905, 1000, 1032, 1047, 1135, 1250]
labels = [value for index, value in enumerate(labels) if index not in None_index_in_justification]
statements = [value for index, value in enumerate(statements) if index not in None_index_in_justification]
justification = [value for index, value in enumerate(justification) if index not in None_index_in_justification]

##change 6 labels to 2 labels
def to_2_labels(a):
      a_label = [""]*len(a)
      for i in range(len(a)):
          if a[i]=='true':
              a_label[i] = 'true'
          elif a[i]=='mostly-true':
              a_label[i] = 'true'
          elif a[i]=='half-true':
              a_label[i] = 'true'
          elif a[i]=='barely-true':
              a_label[i] = 'false'
          elif a[i]=='false':
              a_label[i] = 'true'
          elif a[i]=='pants-fire':
              a_label[i] = 'true'
          else:
              print('Incorrect label')
      return a_label



## concatenate evidence and claim
def concat_evidence_claim(evidence, claim):
      evidence_claim = [""]*len(claim)
      for i in range(len(claim)):
        evidence_claim[i] = evidence[i] + ' ' + claim[i]
      return evidence_claim
def tsv_to_jsonl(a_label, evidence_claim):
    jsonl_file_path = '/content/drive/MyDrive/fake_news_detection/data/liar_plus_test.jsonl'

    with jsonlines.open(jsonl_file_path, mode='w') as writer:
      for i in range(len(evidence_claim)):
        if len(evidence_claim[i]) < 800:
          data = {"label": a_label[i], "claim": evidence_claim[i]}
          writer.write(data)

labels_2 = to_2_labels(labels)
evidence_claim = concat_evidence_claim(justification, statements)

tsv_to_jsonl(labels_2, evidence_claim)




In [5]:
cd ./mlm

/content/drive/MyDrive/fake_news_detection/mlm


In [6]:
import torch
torch.cuda.empty_cache()

In [ ]:
!python main.py --train_data_file='/content/drive/MyDrive/fake_news_detection/data/liar_plus_test.jsonl' --output_eval_file='/content/drive/MyDrive/fake_news_detection/ppl_results/liar_plus_test.npy' --model_name='nghuyong/ernie-2.0-base-en'

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '
Some weights of ErnieForMaskedLMOptimized were not initialized from the model checkpoint at nghuyong/ernie-2.0-base-en and are newly initialized: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Evaluating  /content/drive/MyDrive/fake_news_detection/data/liar_plus_test.jsonl
100% 1120/1120 [04:28<00:00,  4.17it/s]


In [ ]:

import random
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, recall_score, precision_score, \
    confusion_matrix

def get_metric(objs, ppl_threshold, is_print=True, for_excel=False):

    preds = ['true' if float(obj['perplexity']) < ppl_threshold else 'false' for obj in objs]
    golds = [obj['label'] for obj in objs]
    acc = accuracy_score(golds, preds)
    #     tn, fp, fn, tp = confusion_matrix(golds, preds).ravel()
    false_positive_count = 0
    for i in range(len(golds)):
        if golds[i] != preds[i]:
            if golds[i] == 'false':
                false_positive_count += 1

    f1_binary = f1_score(golds, preds, pos_label='false', average='binary')
    f1_macro = f1_score(golds, preds, pos_label='false', average='macro')
    recall = recall_score(golds, preds, pos_label='false', average='binary')
    precision = precision_score(golds, preds, pos_label='false', average='binary')

    if is_print:
        print(
            "TH: {}, Acc: {:.4f}, F1-macro: {:.4f},  F1-binary: {:.4f}, Recall:{:.4f}, Precision: {:.4f}, FN Count: {}". \
            format(ppl_threshold, acc, f1_macro, f1_binary, recall, precision, false_positive_count))
    if for_excel:
        print("for excel sheet: \n {},{},{},{},{},{}" \
              .format(acc, f1_macro, f1_binary, recall, false_positive_count, ppl_threshold))
    return {'acc': acc, 'f1_macro': f1_macro, 'f1_binary': f1_binary, \
            'recall': recall, 'fp': false_positive_count, 'th': ppl_threshold}



data_path = '/content/drive/MyDrive/fake_news_detection/data/liar_plus_test.jsonl'
ppl_path = "/content/drive/MyDrive/fake_news_detection/ppl_results/liar_plus_test.jsonl"

with jsonlines.open(data_path) as reader:
    og_objs = [obj for obj in reader]

with jsonlines.open(ppl_path) as reader:
    ppl = [obj for obj in reader]

perplexity_with_true = []
perplexity_with_false = []


for i in range(len(ppl)):
  data = {"label": og_objs[i]["label"], "perplexity": abs(ppl[i]["perplexity"])}
  if data["label"] == "true":
    perplexity_with_true.append(data)
  else:
    perplexity_with_false.append(data)
print("true_perplexity: ", len(perplexity_with_true), ", false_perplexity: ", len(perplexity_with_false))

k = [2, 10, 50]

thresholds = [i for i in range(1500)]


train_set = []
test_set = []
for i in range(3):
    random.shuffle(perplexity_with_true)
    random.shuffle(perplexity_with_false)

    for j in range(int(k[i]/2)):
      train_set.append(perplexity_with_true[j])
      train_set.append(perplexity_with_false[j])
    for j in range(int(k[i]/2) + 1, len(perplexity_with_false)):
      test_set.append(perplexity_with_true[j])
      test_set.append(perplexity_with_false[j])


    results = []
    for ppl_th in thresholds:
        result = get_metric(train_set, ppl_th, False)
        results.append(result)

        f1_macros = [o['f1_macro'] for o in results]
        max_macro = max(f1_macros)

    optimal_ths = []
    for r in results:
        if r['f1_macro'] == max_macro:
            optimal_th = r['th']
            get_metric(train_set, optimal_th, True)
            optimal_ths.append(optimal_th)
    print(optimal_ths)
    opt_th = np.mean(optimal_ths)
    print(opt_th)
    result_on_test = get_metric(test_set, opt_th, True)

    log_path = '/content/drive/MyDrive/fake_news_detection/results/{}_{}.txt'.format("liar_plus_test", k)

    with open(log_path , "a") as writer:
        writer.write("\n%s\t%s" % ( k[i], result_on_test))


true_perplexity:  947 , false_perplexity:  173


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'false') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'false') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'false') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'false') is ignored when average != 'binary' (got 'macro'). You may use labels=[

TH: 0, Acc: 0.5000, F1-macro: 0.3333,  F1-binary: 0.6667, Recall:1.0000, Precision: 0.5000, FN Count: 0
TH: 1, Acc: 0.5000, F1-macro: 0.3333,  F1-binary: 0.6667, Recall:1.0000, Precision: 0.5000, FN Count: 0
TH: 2, Acc: 0.5000, F1-macro: 0.3333,  F1-binary: 0.6667, Recall:1.0000, Precision: 0.5000, FN Count: 0
TH: 3, Acc: 0.5000, F1-macro: 0.3333,  F1-binary: 0.6667, Recall:1.0000, Precision: 0.5000, FN Count: 0
TH: 4, Acc: 0.5000, F1-macro: 0.3333,  F1-binary: 0.6667, Recall:1.0000, Precision: 0.5000, FN Count: 0
TH: 5, Acc: 0.5000, F1-macro: 0.3333,  F1-binary: 0.6667, Recall:1.0000, Precision: 0.5000, FN Count: 0
TH: 6, Acc: 0.5000, F1-macro: 0.3333,  F1-binary: 0.6667, Recall:1.0000, Precision: 0.5000, FN Count: 0
TH: 7, Acc: 0.5000, F1-macro: 0.3333,  F1-binary: 0.6667, Recall:1.0000, Precision: 0.5000, FN Count: 0
TH: 8, Acc: 0.5000, F1-macro: 0.3333,  F1-binary: 0.6667, Recall:1.0000, Precision: 0.5000, FN Count: 0
TH: 9, Acc: 0.5000, F1-macro: 0.3333,  F1-binary: 0.6667, Recall

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'false') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'false') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'false') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'false') is ignored when average != 'binary' (got 'macro'). You may use labels=[

TH: 28, Acc: 0.5000, F1-macro: 0.3333,  F1-binary: 0.6667, Recall:1.0000, Precision: 0.5000, FN Count: 0
TH: 29, Acc: 0.5000, F1-macro: 0.3333,  F1-binary: 0.6667, Recall:1.0000, Precision: 0.5000, FN Count: 0
TH: 30, Acc: 0.5000, F1-macro: 0.3333,  F1-binary: 0.6667, Recall:1.0000, Precision: 0.5000, FN Count: 0
TH: 31, Acc: 0.5000, F1-macro: 0.3333,  F1-binary: 0.6667, Recall:1.0000, Precision: 0.5000, FN Count: 0
TH: 32, Acc: 0.5000, F1-macro: 0.3333,  F1-binary: 0.6667, Recall:1.0000, Precision: 0.5000, FN Count: 0
TH: 33, Acc: 0.5000, F1-macro: 0.3333,  F1-binary: 0.6667, Recall:1.0000, Precision: 0.5000, FN Count: 0
TH: 34, Acc: 0.5000, F1-macro: 0.3333,  F1-binary: 0.6667, Recall:1.0000, Precision: 0.5000, FN Count: 0
TH: 35, Acc: 0.5000, F1-macro: 0.3333,  F1-binary: 0.6667, Recall:1.0000, Precision: 0.5000, FN Count: 0
TH: 36, Acc: 0.5000, F1-macro: 0.3333,  F1-binary: 0.6667, Recall:1.0000, Precision: 0.5000, FN Count: 0
TH: 37, Acc: 0.5000, F1-macro: 0.3333,  F1-binary: 0.66

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'false') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'false') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'false') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'false') is ignored when average != 'binary' (got 'macro'). You may use labels=[

TH: 780, Acc: 0.6667, F1-macro: 0.6250,  F1-binary: 0.7500, Recall:1.0000, Precision: 0.6000, FN Count: 0
TH: 781, Acc: 0.6667, F1-macro: 0.6250,  F1-binary: 0.7500, Recall:1.0000, Precision: 0.6000, FN Count: 0
TH: 782, Acc: 0.6667, F1-macro: 0.6250,  F1-binary: 0.7500, Recall:1.0000, Precision: 0.6000, FN Count: 0
TH: 783, Acc: 0.6667, F1-macro: 0.6250,  F1-binary: 0.7500, Recall:1.0000, Precision: 0.6000, FN Count: 0
TH: 784, Acc: 0.6667, F1-macro: 0.6250,  F1-binary: 0.7500, Recall:1.0000, Precision: 0.6000, FN Count: 0
TH: 785, Acc: 0.6667, F1-macro: 0.6250,  F1-binary: 0.7500, Recall:1.0000, Precision: 0.6000, FN Count: 0
TH: 786, Acc: 0.6667, F1-macro: 0.6250,  F1-binary: 0.7500, Recall:1.0000, Precision: 0.6000, FN Count: 0
TH: 787, Acc: 0.6667, F1-macro: 0.6250,  F1-binary: 0.7500, Recall:1.0000, Precision: 0.6000, FN Count: 0
TH: 788, Acc: 0.6667, F1-macro: 0.6250,  F1-binary: 0.7500, Recall:1.0000, Precision: 0.6000, FN Count: 0
TH: 789, Acc: 0.6667, F1-macro: 0.6250,  F1-bi

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'false') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'false') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'false') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'false') is ignored when average != 'binary' (got 'macro'). You may use labels=[

TH: 1104, Acc: 0.5484, F1-macro: 0.5479,  F1-binary: 0.5625, Recall:0.5806, Precision: 0.5455, FN Count: 13
TH: 1105, Acc: 0.5484, F1-macro: 0.5479,  F1-binary: 0.5625, Recall:0.5806, Precision: 0.5455, FN Count: 13
TH: 1106, Acc: 0.5484, F1-macro: 0.5479,  F1-binary: 0.5625, Recall:0.5806, Precision: 0.5455, FN Count: 13
TH: 1107, Acc: 0.5484, F1-macro: 0.5479,  F1-binary: 0.5625, Recall:0.5806, Precision: 0.5455, FN Count: 13
TH: 1108, Acc: 0.5484, F1-macro: 0.5479,  F1-binary: 0.5625, Recall:0.5806, Precision: 0.5455, FN Count: 13
[1104, 1105, 1106, 1107, 1108]
1106.0
TH: 1106.0, Acc: 0.4969, F1-macro: 0.4969,  F1-binary: 0.4938, Recall:0.4907, Precision: 0.4969, FN Count: 247


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'false') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [7]:
!python main.py --train_data_file='/content/drive/MyDrive/fake_news_detection/data/fever_test.jsonl' --output_eval_file='/content/drive/MyDrive/fake_news_detection/ppl_results/fever_test.npy' --model_name='nghuyong/ernie-2.0-base-en'

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
Moving 0 files to the new cache system
0it [00:00, ?it/s]
Some weights of ErnieForMaskedLMOptimized were not initialized from the model checkpoint at nghuyong/ernie-2.0-base-en and are newly initialized: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
You should probably TRAIN this model on a down-stream task to be abl

In [8]:
import random
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, recall_score, precision_score, \
    confusion_matrix

def get_metric(objs, ppl_threshold, is_print=True, for_excel=False):

    preds = ['VERIFIABLE' if float(obj['perplexity']) < ppl_threshold else 'NOT VERIFIABLE' for obj in objs]
    golds = [obj["verifiable"] for obj in objs]
    acc = accuracy_score(golds, preds)
    #     tn, fp, fn, tp = confusion_matrix(golds, preds).ravel()
    false_positive_count = 0
    for i in range(len(golds)):
        if golds[i] != preds[i]:
            if golds[i] == 'NOT ENOUGH INFO':
                false_positive_count += 1

    f1_binary = f1_score(golds, preds, pos_label='NOT VERIFIABLE', average='binary')
    f1_macro = f1_score(golds, preds, pos_label='NOT VERIFIABLE', average='macro')
    recall = recall_score(golds, preds, pos_label='NOT VERIFIABLE', average='binary')
    precision = precision_score(golds, preds, pos_label='NOT VERIFIABLE', average='binary')

    if is_print:
        print(
            "TH: {}, Acc: {:.4f}, F1-macro: {:.4f},  F1-binary: {:.4f}, Recall:{:.4f}, Precision: {:.4f}, FN Count: {}". \
            format(ppl_threshold, acc, f1_macro, f1_binary, recall, precision, false_positive_count))
    if for_excel:
        print("for excel sheet: \n {},{},{},{},{},{}" \
              .format(acc, f1_macro, f1_binary, recall, false_positive_count, ppl_threshold))
    return {'acc': acc, 'f1_macro': f1_macro, 'f1_binary': f1_binary, \
            'recall': recall, 'fp': false_positive_count, 'th': ppl_threshold}



data_path = '/content/drive/MyDrive/fake_news_detection/data/fever_test.jsonl'
ppl_path = "/content/drive/MyDrive/fake_news_detection/ppl_results/fever_test.jsonl"

with jsonlines.open(data_path) as reader:
    og_objs = [obj for obj in reader]

with jsonlines.open(ppl_path) as reader:
    ppl = [obj for obj in reader]

perplexity_with_true = []
perplexity_with_false = []


for i in range(len(ppl)):
  data = {"verifiable": og_objs[i]["verifiable"], "perplexity": abs(ppl[i]["perplexity"])}
  if data["verifiable"] == "VERIFIABLE":
    perplexity_with_true.append(data)
  else:
    perplexity_with_false.append(data)
print("true_perplexity: ", len(perplexity_with_true), ", false_perplexity: ", len(perplexity_with_false))

k = [2, 10, 50]

thresholds = [i for i in range(1500)]


train_set = []
test_set = []
for i in range(3):
    random.shuffle(perplexity_with_true)
    random.shuffle(perplexity_with_false)

    for j in range(int(k[i]/2)):
      train_set.append(perplexity_with_true[j])
      train_set.append(perplexity_with_false[j])
    for j in range(int(k[i]/2) + 1, len(perplexity_with_false)):
      test_set.append(perplexity_with_true[j])
      test_set.append(perplexity_with_false[j])


    results = []
    for ppl_th in thresholds:
        result = get_metric(train_set, ppl_th, False)
        results.append(result)

        f1_macros = [o['f1_macro'] for o in results]
        max_macro = max(f1_macros)

    optimal_ths = []
    for r in results:
        if r['f1_macro'] == max_macro:
            optimal_th = r['th']
            get_metric(train_set, optimal_th, True)
            optimal_ths.append(optimal_th)
    print(optimal_ths)
    opt_th = np.mean(optimal_ths)
    print(opt_th)
    result_on_test = get_metric(test_set, opt_th, True)

    log_path = '/content/drive/MyDrive/fake_news_detection/results/{}_{}.txt'.format("fever_test", k)

    with open(log_path , "a") as writer:
        writer.write("\n%s\t%s" % ( k[i], result_on_test))


Output hidden; open in https://colab.research.google.com to view.

/usr/local/lib/python3.10/dist-packages/gluonnlp/model/bert.py

974 line pretrained_allow_missing=True로 바꿈